In [21]:
import pandas as pd
import numpy as np

teams = pd.read_csv('men_teams_historical.csv')
teams = teams[teams['year'] >= 2015]
print(teams.shape)

five_man_lineups = pd.read_csv('men_lineups_5man_historical.csv')
print(five_man_lineups.shape)

three_man_lineups = pd.read_csv('men_lineups_3man_historical.csv')
print(three_man_lineups.shape)

kp_ratings = pd.read_csv('men_kenpom_ratings_historical.csv')
kp_ratings = kp_ratings[['kenpom_id', 'kenpom_off', 'kenpom_def', 'kenpom_rtg']]
print(kp_ratings.shape)

kp_roster = pd.read_csv('men_kenpom_roster_historical.csv')
kp_roster = kp_roster.drop(columns=['year', 'team'])
print(kp_roster.shape)

bt_ratings = pd.read_csv('men_torvik_ratings_historical.csv')
bt_ratings = bt_ratings.drop(columns=['year', 'team'])
print(bt_ratings.shape)

bt_splits = pd.read_csv('men_torvik_splits_historical.csv')
bt_splits = bt_splits.drop(columns=['year', 'team'])
print(bt_splits.shape)

(748, 10)
(748, 6)
(748, 6)
(3914, 4)
(3914, 20)
(3904, 27)
(3904, 17)


In [22]:
df = teams.merge(five_man_lineups, left_on='team_id', right_on='team_id', how='inner')
df = df.merge(three_man_lineups, left_on='team_id', right_on='team_id', how='inner')
df = df.merge(kp_ratings, left_on='kenpom_id', right_on='kenpom_id', how='inner')
df = df.merge(kp_roster, left_on='kenpom_id', right_on='kenpom_id', how='inner')
df = df.merge(bt_ratings, left_on='torvik_id', right_on='torvik_id', how='inner')
df = df.merge(bt_splits, left_on='torvik_id', right_on='torvik_id', how='inner')
print(df.shape)


(748, 84)


# Feature Engineering

In [23]:
# ----------------------------------------------------------------------------
# INVERT DEFENSIVE PERCENTAGES (100 - value)
# ----------------------------------------------------------------------------
df['efgd%'] = 100 - df['efgd%']
df['2p%d'] = 100 - df['2p%d']
df['3p%d'] = 100 - df['3p%d']
df['ft%d'] = 100 - df['ft%d']
df['def_dunk_fg%'] = 100 - df['def_dunk_fg%']
df['def_close2_fg%'] = 100 - df['def_close2_fg%']
df['def_far2_fg%'] = 100 - df['def_far2_fg%']
df['def_3pt_fg%'] = 100 - df['def_3pt_fg%']
df['ast%d'] = 100 - df['ast%d']

# ----------------------------------------------------------------------------
# INVERT OFFENSIVE "BAD" METRICS (100 - value)
# ----------------------------------------------------------------------------
df['tor'] = 100 - df['tor']  # Higher = better ball security
df['blked%'] = 100 - df['blked%']  # Higher = better finishing

# ----------------------------------------------------------------------------
# INVERT DEFENSIVE EFFICIENCY RATINGS (200 - value)
# ----------------------------------------------------------------------------
df['kenpom_def'] = 200 - df['kenpom_def']
df['torvik_def'] = 200 - df['torvik_def']

# ----------------------------------------------------------------------------
# INVERT FREE THROW RATE DEFENSE (100 - value)
# ----------------------------------------------------------------------------
df['ftrd'] = 100 - df['ftrd']

# ----------------------------------------------------------------------------
# INVERT LINEUP DEFENSIVE METRICS (50 - value)
# ----------------------------------------------------------------------------
df['5man_dprpg'] = 50 - df['5man_dprpg']
df['3man_dprpg'] = 50 - df['3man_dprpg']

In [24]:
# ----------------------------------------------------------------------------
# 1. EFFICIENCY MARGINS (Both offensive AND defensive pairs)
# ----------------------------------------------------------------------------
# These create net advantage metrics comparing offense vs defense
# All are now simple subtraction since features are inverted

# OFFENSIVE MARGINS

# 1. Net eFG Margin
# Calculation: Team eFG% - Opponent eFG% allowed
# Higher = shoots better than opponents
df['net_efg_margin'] = df['efg%'] - df['efgd%']

# 2. Net Turnover Margin
# Calculation: Opponent TOs forced - Own TOs committed
# Higher = forces more TOs than commits
df['net_turnover_margin'] = df['tord'] - df['tor']

# 3. Net Rebounding Margin
# Calculation: (ORB% + DRB%) - 100
# Higher = dominates boards on both ends
df['net_rebounding_margin'] = (df['orb%'] + df['drb%']) - 100

# 4. Net FTR Margin
# Calculation: Team FTR - Opponent FTR allowed
# Higher = gets to line more than opponents
df['net_ftr_margin'] = df['ftr'] - df['ftrd']

# DEFENSIVE MARGINS (inverse of offensive for pairing)

# 1. Defensive Net eFG Margin
# Calculation: Opponent eFG% allowed - Team eFG%
# Higher = better defensive eFG% than offensive
df['def_net_efg_margin'] = df['efgd%'] - df['efg%']

# 2. Defensive Net Turnover Margin
# Calculation: Own TOs - Opponent TOs forced
# Higher = better ball security relative to forcing TOs
df['def_net_turnover_margin'] = df['tor'] - df['tord']

# 3. Defensive Net Rebounding Margin
# Calculation: 100 - (ORB% + DRB%)
# Higher = limits opponent rebounding advantage
df['def_net_rebounding_margin'] = 100 - (df['orb%'] + df['drb%'])

# 4. Defensive Net FTR Margin
# Calculation: Opponent FTR allowed - Team FTR
# Higher = keeps opponents off line relative to own FTR
df['def_net_ftr_margin'] = df['ftrd'] - df['ftr']

In [25]:
# ----------------------------------------------------------------------------
# 2. SHOT QUALITY (Weighted efficiency by zone - BOTH sides)
# ----------------------------------------------------------------------------
# OFFENSIVE SHOT QUALITY

# 1. Rim Efficiency (dunks + close 2s)
# Calculation: Weighted average of shooting % at rim by volume
# Higher = better finishing at the basket
df['rim_efficiency'] = (df['off_dunk_fg%'] * df['off_dunk_share']/100) + \
                       (df['off_close2_fg%'] * df['off_close2_share']/100)

# 2. Perimeter Efficiency (far 2s + 3s)
# Calculation: Weighted average of shooting % outside paint by volume
# Higher = better jump shooting
df['perimeter_efficiency'] = (df['off_far2_fg%'] * df['off_far2_share']/100) + \
                              (df['off_3pt_fg%'] * df['off_3pt_share']/100)

# 3. Three-Point Volume Efficiency
# Calculation: 3PT% × 3PT volume
# Higher = high volume AND high accuracy from three
df['three_point_volume_efficiency'] = df['off_3pt_fg%'] * df['off_3pt_share'] / 100

# 4. Shot Quality Variance (consistency across zones)
# Calculation: Standard deviation of shooting % across all zones
# Lower variance = more consistent/balanced shooting
df['shot_quality_variance'] = df[['off_dunk_fg%', 'off_close2_fg%', 'off_far2_fg%', 'off_3pt_fg%']].std(axis=1)

# DEFENSIVE SHOT QUALITY (already inverted, so higher = better)

# 1. Defensive Rim Efficiency
# Calculation: Weighted average of opponent shooting % at rim
# Higher = better rim protection (inverted percentages)
df['def_rim_efficiency'] = (df['def_dunk_fg%'] * df['def_dunk_share']/100) + \
                            (df['def_close2_fg%'] * df['def_close2_share']/100)

# 2. Defensive Perimeter Efficiency
# Calculation: Weighted average of opponent perimeter shooting
# Higher = better perimeter defense (inverted percentages)
df['def_perimeter_efficiency'] = (df['def_far2_fg%'] * df['def_far2_share']/100) + \
                                  (df['def_3pt_fg%'] * df['def_3pt_share']/100)

# 3. Defensive Three-Point Volume Efficiency
# Calculation: Opponent 3PT% × opponent 3PT volume
# Higher = limits opponent 3PT impact (inverted percentage)
df['def_three_point_volume_efficiency'] = df['def_3pt_fg%'] * df['def_3pt_share'] / 100

# 4. Defensive Shot Quality Variance
# Calculation: Standard deviation of opponent shooting % across zones
# Lower variance = consistent defense across all zones
df['def_shot_quality_variance'] = df[['def_dunk_fg%', 'def_close2_fg%', 'def_far2_fg%', 'def_3pt_fg%']].std(axis=1)

In [26]:
# ----------------------------------------------------------------------------
# 3. SHOT SELECTION (BOTH sides)
# ----------------------------------------------------------------------------
# OFFENSIVE SHOT SELECTION

# 1. Rim-to-Three Ratio (Moreyball principle)
# Calculation: (Dunks + Close 2s) / Threes
# Higher = more rim attempts relative to threes
# Optimal ~1.0-1.5 (balanced rim and three approach)
df['rim_to_three_ratio'] = (df['off_dunk_share'] + df['off_close2_share']) / (df['off_3pt_share'] + 0.001)

# 2. Mid-Range Reliance (INVERTED - less mid-range = better)
# Calculation: 50 - Far 2PT share
# Higher = takes fewer mid-range shots (more efficient)
df['mid_range_reliance'] = 50 - df['off_far2_share']

# 3. Paint Touch Rate
# Calculation: Dunks + Close 2s share
# Higher = more paint penetration
df['paint_touch_rate'] = df['off_dunk_share'] + df['off_close2_share']

# DEFENSIVE SHOT SELECTION

# 1. Defensive Rim-to-Three Ratio
# Calculation: (Opponent dunks + close 2s) / Opponent threes
# Shows what type of shots defense allows
df['def_rim_to_three_ratio'] = (df['def_dunk_share'] + df['def_close2_share']) / (df['def_3pt_share'] + 0.001)

# 2. Defensive Mid-Range Reliance
# Calculation: Opponent far 2PT share
# Higher = forces more mid-range shots (good defense)
df['def_mid_range_reliance'] = df['def_far2_share']

# 3. Defensive Paint Touch Rate (INVERTED)
# Calculation: 100 - (Opponent dunks + close 2s)
# Higher = limits opponent paint touches
df['def_paint_touch_rate'] = 100 - (df['def_dunk_share'] + df['def_close2_share'])

In [27]:
# ----------------------------------------------------------------------------
# 4. DEPTH & ROTATION
# ----------------------------------------------------------------------------
# These measure scoring distribution and bench contribution

# 1. Top 5 Scoring Concentration (INVERTED - less concentration = better depth)
# Calculation: 100 - (sum of top 5 scorers' shares)
# Higher = more balanced/distributed scoring
df['top5_scoring_concentration'] = 100 - (df['pts_from_5'] + df['pts_from_4'] + df['pts_from_3'] + \
                                           df['pts_from_2'] + df['pts_from_1'])

# 2. Top 5 Offensive Rebounding Concentration (INVERTED)
# Calculation: 100 - (sum of top 5 offensive rebounders)
# Higher = offensive rebounding by committee
df['top5_rebounding_concentration'] = 100 - (df['or_from_5'] + df['or_from_4'] + df['or_from_3'] + \
                                              df['or_from_2'] + df['or_from_1'])

# 3. Top 5 Defensive Rebounding Concentration (INVERTED)
# Calculation: 100 - (sum of top 5 defensive rebounders)
# Higher = defensive rebounding by committee
df['top5_def_rebounding_concentration'] = 100 - (df['dr_from_5'] + df['dr_from_4'] + df['dr_from_3'] + \
                                                  df['dr_from_2'] + df['dr_from_1'])

# 4. Bench Scoring Ratio
# Calculation: Bench scoring percentage (already in dataset)
# Higher = better bench production
df['bench_scoring_ratio'] = df['bench']

# 5. Rotation Balance (evenness of top 5 scoring)
# Calculation: 100 - standard deviation of top 5 scorers
# Higher = more balanced scoring among starters
pts_cols = ['pts_from_5', 'pts_from_4', 'pts_from_3', 'pts_from_2', 'pts_from_1']
df['rotation_balance'] = 100 - df[pts_cols].std(axis=1)

In [28]:
# ----------------------------------------------------------------------------
# 5. PACE & TEMPO (BOTH sides)
# ----------------------------------------------------------------------------
# 1. Tempo Advantage
# Calculation: Adjusted tempo - 68 (national average)
# Positive = plays faster, Negative = plays slower
df['tempo_advantage'] = df['adj_tempo'] - 68

# 2. Effective Possession Rate (OFFENSIVE)
# Calculation: Ball security rate × shooting efficiency
# Higher = maximizes possessions by protecting ball AND shooting well
df['effective_possession_rate'] = (df['tor'] / 100) * df['efg%']

# 3. Defensive Effective Possession Rate
# Calculation: Opponent ball security × opponent shooting
# Higher = limits opponent effectiveness
df['def_effective_possession_rate'] = (df['tord'] / 100) * df['efgd%']

In [29]:
# ----------------------------------------------------------------------------
# 6. VERSATILITY (BOTH sides)
# ----------------------------------------------------------------------------
# OFFENSIVE VERSATILITY

# 1. Offensive Versatility Score
# Calculation: Inverse of coefficient of variation across key metrics
# Higher = well-rounded offense (2P%, 3P%, AST%, ORB%)
off_metrics = df[['2p%', '3p%', 'ast%', 'orb%']].values
df['offensive_versatility_score'] = 100 / (np.std(off_metrics, axis=1) / (np.mean(off_metrics, axis=1) + 0.001) + 1)

# 2. Assist-to-Usage Ratio
# Calculation: Assist % / (100 - Opponent TO%)
# Higher = good ball movement relative to possessions
df['assist_to_usage_ratio'] = df['ast%'] / (100 - df['tord'] + 0.001)

# DEFENSIVE VERSATILITY

# 1. Defensive Versatility Score
# Calculation: Inverse of coefficient of variation across defensive metrics
# Higher = well-rounded defense (2P%d, 3P%d, AST%d, DRB%)
def_metrics = df[['2p%d', '3p%d', 'ast%d', 'drb%']].values
df['defensive_versatility_score'] = 100 / (np.std(def_metrics, axis=1) / (np.mean(def_metrics, axis=1) + 0.001) + 1)

# 2. Defensive Assist Suppression
# Calculation: Opponent assist % / (100 - Own TO%)
# Higher = limits opponent ball movement
df['def_assist_suppression'] = df['ast%d'] / (100 - df['tor'] + 0.001)

# TEAM METRICS (same for both offense/defense)

# Size-Speed Index
# Calculation: Height × Tempo / 100
# Higher = big AND fast (rare, valuable combination)
df['size_speed_index'] = (df['height'] * df['adj_tempo']) / 100
df['def_size_speed_index'] = df['size_speed_index']

In [30]:
# ----------------------------------------------------------------------------
# 7. CLUTCH & PRESSURE (BOTH sides)
# ----------------------------------------------------------------------------
# OFFENSIVE CLUTCH

# 1. Free Throw Advantage
# Calculation: (Own FT% × FTR) - (Opponent FT% × FTR allowed)
# Higher = gains more from free throw line
df['free_throw_advantage'] = (df['ft%'] * df['ftr']) - (df['ft%d'] * df['ftrd'])

# 2. Block Efficiency
# Calculation: Blocks given / Blocks received
# Higher = blocks more than gets blocked
df['block_efficiency'] = df['blk%'] / (df['blked%'] + 0.001)

# 3. Experience-Weighted Production
# Calculation: Best 5-man offensive BPM × Team experience
# Higher = experienced AND productive lineup
df['experience_weighted_production'] = df['5man_obpm'] * df['experience']

# DEFENSIVE CLUTCH

# 1. Defensive Free Throw Advantage
# Calculation: (Opponent FT% × FTR) - (Own FT% × FTR)
# Higher = limits opponent FT advantage
df['def_free_throw_advantage'] = (df['ft%d'] * df['ftrd']) - (df['ft%'] * df['ftr'])

# 2. Defensive Block Efficiency
# Calculation: Blocks received / Blocks given
# Higher = absorbs blocks well relative to blocking
df['def_block_efficiency'] = df['blked%'] / (df['blk%'] + 0.001)

# 3. Defensive Experience Impact
# Calculation: Best 5-man defensive BPM × Team experience
# Higher = experienced defensive lineup
df['def_experience_impact'] = df['5man_dbpm'] * df['experience']

In [31]:
# ----------------------------------------------------------------------------
# 8. ADVANCED COMPOSITES (BOTH sides)
# ----------------------------------------------------------------------------
# OFFENSIVE COMPOSITE

# Four Factors Composite (Dean Oliver's research-backed weights)
# Calculation: 40% eFG + 25% TOR + 20% ORB + 15% FTR
# Higher = better overall offensive profile
df['four_factors_composite'] = (0.40 * df['efg%']) + \
                                (0.25 * df['tor']) + \
                                (0.20 * df['orb%']) + \
                                (0.15 * df['ftr'])

# DEFENSIVE COMPOSITE

# Defensive Four Factors Composite
# Calculation: 40% eFGd + 25% TORd + 20% DRB + 15% FTRd
# Higher = better overall defensive profile
df['def_four_factors_composite'] = (0.40 * df['efgd%']) + \
                                    (0.25 * df['tord']) + \
                                    (0.20 * df['drb%']) + \
                                    (0.15 * df['ftrd'])

# TEAM COMPOSITE

# Elite Outcome Probability
# Calculation: Count of metrics in top 25%
# Higher = elite in more areas (championship teams typically 3-5)
df['elite_outcome_probability'] = 0
df['elite_outcome_probability'] += (df['efg%'] >= df['efg%'].quantile(0.75)).astype(int)
df['elite_outcome_probability'] += (df['tor'] >= df['tor'].quantile(0.75)).astype(int)
df['elite_outcome_probability'] += (df['orb%'] >= df['orb%'].quantile(0.75)).astype(int)
df['elite_outcome_probability'] += (df['3p%'] >= df['3p%'].quantile(0.75)).astype(int)
df['elite_outcome_probability'] += (df['drb%'] >= df['drb%'].quantile(0.75)).astype(int)

In [32]:
# ----------------------------------------------------------------------------
# 9. SYNERGY (BOTH sides where applicable)
# ----------------------------------------------------------------------------
# OFFENSIVE SYNERGY

# 1. Shooting Variance Resilience
# Calculation: Inverse of coefficient of variation in shooting %
# Higher = consistent shooting across all zones
shooting_cols = ['off_dunk_fg%', 'off_close2_fg%', 'off_far2_fg%', 'off_3pt_fg%']
df['shooting_variance_resilience'] = 100 / (df[shooting_cols].std(axis=1) / (df[shooting_cols].mean(axis=1) + 0.001) + 1)

# 2. Lineup Depth Quality (INVERTED - smaller drop-off = better)
# Calculation: -(5-man OBPM - 3-man OBPM)
# Higher = less production drop from best 5-man to 3-man
df['lineup_depth_quality'] = -(df['5man_obpm'] - df['3man_obpm'])

# DEFENSIVE SYNERGY

# Defensive Lineup Depth Quality (INVERTED)
# Calculation: -(5-man DBPM - 3-man DBPM)
# Higher = less defensive drop-off (better depth)
df['def_lineup_depth_quality'] = -(df['5man_dbpm'] - df['3man_dbpm'])

# TEAM SYNERGY

# Offense-Defense Balance
# Calculation: Penalty for deviation from optimal 1.25 ratio
# Higher = more balanced team (optimal offense/defense ratio)
balance_raw = df['kenpom_off'] / (df['kenpom_def'] + 0.001)
df['offense_defense_balance'] = 10 - abs(balance_raw - 1.25) * 10

In [33]:
df.shape

(748, 133)

In [34]:
df.head()

,team_id,year,team,region,seed,finish,weekend,conference,kenpom_id,torvik_id,...,def_free_throw_advantage,def_block_efficiency,def_experience_impact,four_factors_composite,def_four_factors_composite,elite_outcome_probability,shooting_variance_resilience,lineup_depth_quality,def_lineup_depth_quality,offense_defense_balance
0,2015 Duke,2015,Duke,South,1,Champion,3,ACC,2015 Duke,2015 Duke,...,-494.78,10.290979,13.356,57.07,43.000,4,67.520238,-6.8,-6.3,9.327345
1,2015 Kentucky,2015,Kentucky,Midwest,1,Final Four,3,SEC,2015 Kentucky,2015 Kentucky,...,-768.36,5.119287,16.425,56.27,46.150,2,67.013975,-6.4,-5.2,7.939524
2,2015 Michigan St.,2015,Michigan St.,East,7,Final Four,3,Big 10,2015 Michigan St.,2015 Michigan St.,...,-256.18,7.022195,23.865,53.75,40.515,2,69.585951,-6.3,-5.7,8.550415
3,2015 Wisconsin,2015,Wisconsin,West,1,Runner Up,3,Big 10,2015 Wisconsin,2015 Wisconsin,...,-199.37,10.227149,38.064,55.46,42.070,2,70.536049,-7.7,-9.5,9.384012
4,2015 Arizona,2015,Arizona,West,2,Elite Eight,2,Pac 12,2015 Arizona,2015 Arizona,...,-1330.33,9.185374,28.860,56.15,41.570,1,70.135891,-3.1,-5.3,7.862727


# Matchups

In [35]:
games = pd.read_csv('men_games_historical.csv')
print(games.shape)
games.head()

(2642, 15)


,game_id,side,id,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,high_bracket_score,low_bracket_score,point_diff,point_total,win
0,20041101a,a,1101,2004,East,First Round,1,2004 Saint Joseph's,16,2004 Liberty,82,63,19,145,1
1,20041102a,a,1102,2004,East,First Round,8,2004 Texas Tech,9,2004 Charlotte,76,73,3,149,1
2,20041103a,a,1103,2004,East,First Round,5,2004 Florida,12,2004 Manhattan,60,75,-15,135,0
3,20041104a,a,1104,2004,East,First Round,4,2004 Wake Forest,13,2004 VCU,79,78,1,157,1
4,20041105a,a,1105,2004,East,First Round,6,2004 Wisconsin,11,2004 Richmond,76,64,12,140,1


In [39]:
matchups = games.merge(
    df,
    left_on='high_bracket_team',
    right_on='team_id',
    how='inner',
    suffixes=('', '_DROP')
)

for col in df.columns:
    if col in matchups.columns and col not in games.columns:
        matchups.rename(columns={col: f'high_{col}'}, inplace=True)

matchups = matchups.merge(
    df,
    left_on='low_bracket_team',
    right_on='team_id',
    how='inner',
    suffixes=('', '_DROP')
)
# Rename all team feature columns with "low_" prefix
for col in df.columns:
    if col in matchups.columns and col not in games.columns and not col.startswith('high_'):
        matchups.rename(columns={col: f'low_{col}'}, inplace=True)

In [40]:
matchups

,game_id,side,id,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,...,low_def_free_throw_advantage,low_def_block_efficiency,low_def_experience_impact,low_four_factors_composite,low_def_four_factors_composite,low_elite_outcome_probability,low_shooting_variance_resilience,low_lineup_depth_quality,low_def_lineup_depth_quality,low_offense_defense_balance
0,20151101a,a,1101,2015,Midwest,First Round,1,2015 Kentucky,16,2015 Hampton,...,-1136.35,8.323303,-14.328000,51.265,40.720,1,67.902456,1.9,4.0,7.561792
1,20151102a,a,1102,2015,Midwest,First Round,8,2015 Cincinnati,9,2015 Purdue,...,-1047.80,6.467626,16.560000,53.895,41.695,2,67.159623,-1.8,-5.9,8.243194
2,20151103a,a,1103,2015,Midwest,First Round,5,2015 West Virginia,12,2015 Buffalo,...,-816.50,8.123175,9.744000,53.625,41.560,2,66.528036,-1.0,-2.7,8.524540
3,20151104a,a,1104,2015,Midwest,First Round,4,2015 Maryland,13,2015 Valparaiso,...,-495.56,7.038214,21.584000,53.290,42.305,2,70.012196,2.2,-9.5,7.719222
4,20151105a,a,1105,2015,Midwest,First Round,6,2015 Butler,11,2015 Texas,...,-440.94,4.524528,13.855000,52.895,42.340,1,69.662969,-3.4,-0.2,8.025447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,20254315b,b,4315,2025,Midwest,Elite Eight,2,2025 Tennessee,1,2025 Houston,...,-171.93,5.911018,55.444896,53.880,43.210,3,70.836023,-4.7,-9.8,8.480801
1250,20254415b,b,4415,2025,West,Elite Eight,3,2025 Texas Tech,1,2025 Florida,...,-377.58,7.430290,46.860672,55.760,41.940,3,69.063664,-6.2,-9.3,9.455728
1251,20255516b,b,5516,2025,Final Four,Final Four,1,2025 Florida,1,2025 Auburn,...,-867.80,5.605721,45.746765,55.815,41.130,4,72.191839,-8.0,-4.9,9.544981
1252,20255517b,b,5517,2025,Final Four,Final Four,1,2025 Houston,1,2025 Duke,...,-390.54,8.518064,41.651085,56.215,43.115,4,71.835538,-8.8,-7.6,9.110913


In [41]:
matchups.columns.to_list()

['game_id',
 'side',
 'id',
 'year',
 'region',
 'round',
 'high_bracket_seed',
 'high_bracket_team',
 'low_bracket_seed',
 'low_bracket_team',
 'high_bracket_score',
 'low_bracket_score',
 'point_diff',
 'point_total',
 'win',
 'high_team_id',
 'year_DROP',
 'high_team',
 'region_DROP',
 'high_seed',
 'high_finish',
 'high_weekend',
 'high_conference',
 'high_kenpom_id',
 'high_torvik_id',
 'high_5man_prpg!',
 'high_5man_dprpg',
 'high_5man_bpm',
 'high_5man_obpm',
 'high_5man_dbpm',
 'high_3man_prpg!',
 'high_3man_dprpg',
 'high_3man_bpm',
 'high_3man_obpm',
 'high_3man_dbpm',
 'high_kenpom_off',
 'high_kenpom_def',
 'high_kenpom_rtg',
 'high_size',
 'high_height',
 'high_experience',
 'high_bench',
 'high_pts_from_5',
 'high_pts_from_4',
 'high_pts_from_3',
 'high_pts_from_2',
 'high_pts_from_1',
 'high_or_from_5',
 'high_or_from_4',
 'high_or_from_3',
 'high_or_from_2',
 'high_or_from_1',
 'high_dr_from_5',
 'high_dr_from_4',
 'high_dr_from_3',
 'high_dr_from_2',
 'high_dr_from_1',

In [42]:
# Create blank differential dataset
matchups_df = pd.DataFrame()

# Copy game info
matchups_df['game_id'] = matchups['game_id']
matchups_df['year'] = matchups['year']
matchups_df['region'] = matchups['region']
matchups_df['round'] = matchups['round']
matchups_df['high_bracket_team'] = matchups['high_bracket_team']
matchups_df['low_bracket_team'] = matchups['low_bracket_team']
matchups_df['high_bracket_seed'] = matchups['high_bracket_seed']
matchups_df['low_bracket_seed'] = matchups['low_bracket_seed']
matchups_df['win'] = matchups['win']

# COMPOSITES (team vs team)
matchups_df['5man_bpm'] = matchups['high_5man_bpm'] - matchups['low_5man_bpm']
matchups_df['3man_bpm'] = matchups['high_3man_bpm'] - matchups['low_3man_bpm']
matchups_df['wab'] = matchups['high_wab'] - matchups['low_wab']
matchups_df['kenpom_rtg'] = matchups['high_kenpom_rtg'] - matchups['low_kenpom_rtg']
matchups_df['torvik_rtg'] = matchups['high_torvik_rtg'] - matchups['low_torvik_rtg']

# HIGH OFFENSE vs LOW DEFENSE
matchups_df['5man_obpm'] = matchups['high_5man_obpm'] - matchups['low_5man_dbpm']
matchups_df['3man_obpm'] = matchups['high_3man_obpm'] - matchups['low_3man_dbpm']
matchups_df['kenpom_off'] = matchups['high_kenpom_off'] - matchups['low_kenpom_def']
matchups_df['torvik_off'] = matchups['high_torvik_off'] - matchups['low_torvik_def']
matchups_df['efg_pct'] = matchups['high_efg%'] - matchups['low_efgd%']
matchups_df['2p_pct'] = matchups['high_2p%'] - matchups['low_2p%d']
matchups_df['3p_pct'] = matchups['high_3p%'] - matchups['low_3p%d']
matchups_df['ft_pct'] = matchups['high_ft%'] - matchups['low_ft%d']
matchups_df['ftr'] = matchups['high_ftr'] - matchups['low_ftrd']
matchups_df['tor'] = matchups['high_tor'] - matchups['low_tord']
matchups_df['ast_pct'] = matchups['high_ast%'] - matchups['low_ast%d']
matchups_df['blk_pct'] = matchups['high_blk%'] - matchups['low_blked%']
matchups_df['off_dunk_fg_pct'] = matchups['high_off_dunk_fg%'] - matchups['low_def_dunk_fg%']
matchups_df['off_close2_fg_pct'] = matchups['high_off_close2_fg%'] - matchups['low_def_close2_fg%']
matchups_df['off_far2_fg_pct'] = matchups['high_off_far2_fg%'] - matchups['low_def_far2_fg%']
matchups_df['off_3pt_fg_pct'] = matchups['high_off_3pt_fg%'] - matchups['low_def_3pt_fg%']
matchups_df['net_efg_margin'] = matchups['high_net_efg_margin'] - matchups['low_def_net_efg_margin']
matchups_df['net_turnover_margin'] = matchups['high_net_turnover_margin'] - matchups['low_def_net_turnover_margin']
matchups_df['net_rebounding_margin'] = matchups['high_net_rebounding_margin'] - matchups['low_def_net_rebounding_margin']
matchups_df['net_ftr_margin'] = matchups['high_net_ftr_margin'] - matchups['low_def_net_ftr_margin']
matchups_df['rim_efficiency'] = matchups['high_rim_efficiency'] - matchups['low_def_rim_efficiency']
matchups_df['perimeter_efficiency'] = matchups['high_perimeter_efficiency'] - matchups['low_def_perimeter_efficiency']
matchups_df['three_point_volume_efficiency'] = matchups['high_three_point_volume_efficiency'] - matchups['low_def_three_point_volume_efficiency']
matchups_df['paint_touch_rate'] = matchups['high_paint_touch_rate'] - matchups['low_def_paint_touch_rate']
matchups_df['effective_possession_rate'] = matchups['high_effective_possession_rate'] - matchups['low_def_effective_possession_rate']
matchups_df['offensive_versatility_score'] = matchups['high_offensive_versatility_score'] - matchups['low_defensive_versatility_score']
matchups_df['assist_to_usage_ratio'] = matchups['high_assist_to_usage_ratio'] - matchups['low_def_assist_suppression']
matchups_df['free_throw_advantage'] = matchups['high_free_throw_advantage'] - matchups['low_def_free_throw_advantage']
matchups_df['block_efficiency'] = matchups['high_block_efficiency'] - matchups['low_def_block_efficiency']
matchups_df['experience_weighted_production'] = matchups['high_experience_weighted_production'] - matchups['low_def_experience_impact']
matchups_df['four_factors_composite'] = matchups['high_four_factors_composite'] - matchups['low_def_four_factors_composite']
matchups_df['lineup_depth_quality'] = matchups['high_lineup_depth_quality'] - matchups['low_def_lineup_depth_quality']

# HIGH DEFENSE vs LOW OFFENSE
matchups_df['5man_dbpm'] = matchups['high_5man_dbpm'] - matchups['low_5man_obpm']
matchups_df['3man_dbpm'] = matchups['high_3man_dbpm'] - matchups['low_3man_obpm']
matchups_df['kenpom_def'] = matchups['high_kenpom_def'] - matchups['low_kenpom_off']
matchups_df['torvik_def'] = matchups['high_torvik_def'] - matchups['low_torvik_off']
matchups_df['efgd_pct'] = matchups['high_efgd%'] - matchups['low_efg%']
matchups_df['2pd_pct'] = matchups['high_2p%d'] - matchups['low_2p%']
matchups_df['3pd_pct'] = matchups['high_3p%d'] - matchups['low_3p%']
matchups_df['ftd_pct'] = matchups['high_ft%d'] - matchups['low_ft%']
matchups_df['ftrd'] = matchups['high_ftrd'] - matchups['low_ftr']
matchups_df['tord'] = matchups['high_tord'] - matchups['low_tor']
matchups_df['astd_pct'] = matchups['high_ast%d'] - matchups['low_ast%']
matchups_df['blked_pct'] = matchups['high_blked%'] - matchups['low_blk%']
matchups_df['def_dunk_fg_pct'] = matchups['high_def_dunk_fg%'] - matchups['low_off_dunk_fg%']
matchups_df['def_close2_fg_pct'] = matchups['high_def_close2_fg%'] - matchups['low_off_close2_fg%']
matchups_df['def_far2_fg_pct'] = matchups['high_def_far2_fg%'] - matchups['low_off_far2_fg%']
matchups_df['def_3pt_fg_pct'] = matchups['high_def_3pt_fg%'] - matchups['low_off_3pt_fg%']
matchups_df['def_net_efg_margin'] = matchups['high_def_net_efg_margin'] - matchups['low_net_efg_margin']
matchups_df['def_net_turnover_margin'] = matchups['high_def_net_turnover_margin'] - matchups['low_net_turnover_margin']
matchups_df['def_net_rebounding_margin'] = matchups['high_def_net_rebounding_margin'] - matchups['low_net_rebounding_margin']
matchups_df['def_net_ftr_margin'] = matchups['high_def_net_ftr_margin'] - matchups['low_net_ftr_margin']
matchups_df['def_rim_efficiency'] = matchups['high_def_rim_efficiency'] - matchups['low_rim_efficiency']
matchups_df['def_perimeter_efficiency'] = matchups['high_def_perimeter_efficiency'] - matchups['low_perimeter_efficiency']
matchups_df['def_three_point_volume_efficiency'] = matchups['high_def_three_point_volume_efficiency'] - matchups['low_three_point_volume_efficiency']
matchups_df['def_paint_touch_rate'] = matchups['high_def_paint_touch_rate'] - matchups['low_paint_touch_rate']
matchups_df['def_effective_possession_rate'] = matchups['high_def_effective_possession_rate'] - matchups['low_effective_possession_rate']
matchups_df['defensive_versatility_score'] = matchups['high_defensive_versatility_score'] - matchups['low_offensive_versatility_score']
matchups_df['def_assist_suppression'] = matchups['high_def_assist_suppression'] - matchups['low_assist_to_usage_ratio']
matchups_df['def_free_throw_advantage'] = matchups['high_def_free_throw_advantage'] - matchups['low_free_throw_advantage']
matchups_df['def_block_efficiency'] = matchups['high_def_block_efficiency'] - matchups['low_block_efficiency']
matchups_df['def_experience_impact'] = matchups['high_def_experience_impact'] - matchups['low_experience_weighted_production']
matchups_df['def_four_factors_composite'] = matchups['high_def_four_factors_composite'] - matchups['low_four_factors_composite']
matchups_df['def_lineup_depth_quality'] = matchups['high_def_lineup_depth_quality'] - matchups['low_lineup_depth_quality']

# TEAM vs TEAM
matchups_df['orb_pct'] = matchups['high_orb%'] - matchups['low_drb%']
matchups_df['drb_pct'] = matchups['high_drb%'] - matchups['low_orb%']
matchups_df['size'] = matchups['high_size'] - matchups['low_size']
matchups_df['height'] = matchups['high_height'] - matchups['low_height']
matchups_df['experience'] = matchups['high_experience'] - matchups['low_experience']
matchups_df['bench'] = matchups['high_bench'] - matchups['low_bench']
matchups_df['adj_tempo'] = matchups['high_adj_tempo'] - matchups['low_adj_tempo']
matchups_df['size_speed_index'] = matchups['high_size_speed_index'] - matchups['low_size_speed_index']
matchups_df['seed'] = matchups['low_bracket_seed'] - matchups['high_bracket_seed']

print(f"Matchups: {matchups_df.shape}")

Matchups: (1254, 87)


In [43]:
matchups_df.to_csv('men_2026_matchups_training.csv')